Mooncoin Market Analysis
========================

Documentation:
- https://github.com/ccxt/ccxt/wiki

In [1]:
import ccxt
from pydash import py_ as _
from datetime import datetime, timedelta
from dateutil.parser import parse
import pandas as pd
import datetime
import numpy as np
import matplotlib
import re

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 8)

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [2]:
bleutrade = ccxt.bleutrade()
bittrex   = ccxt.bittrex()

In [3]:
moonbtc_trades = pd.DataFrame(bleutrade.fetch_trades("MOON/BTC", params={"count": 200}), columns=['amount', 'datetime', 'side', 'info'])
moonbtc_trades['price']      = moonbtc_trades['info'].map(lambda x: float(x['Price']))
moonbtc_trades['btc_amount'] = moonbtc_trades['amount'] * moonbtc_trades['price']
del moonbtc_trades['info']
moonbtc_trades['datetime'] = pd.to_datetime(moonbtc_trades['datetime']) # convert seconds to nanoseconds
moonbtc_trades

            amount            datetime  side         price  btc_amount
0    984119.497245 2017-12-31 18:08:23   buy  2.000000e-08    0.019682
1    505620.502755 2017-12-31 18:08:23   buy  2.000000e-08    0.010112
2     52000.000000 2017-12-31 18:01:14   buy  2.000000e-08    0.001040
3      5000.000000 2017-12-31 18:00:55   buy  2.000000e-08    0.000100
..             ...                 ...   ...           ...         ...
196   14786.000000 2017-12-31 03:37:35  sell  1.000000e-08    0.000148
197   14786.000000 2017-12-31 03:37:35  sell  1.000000e-08    0.000148
198   14786.000000 2017-12-31 03:37:35  sell  1.000000e-08    0.000148
199   14786.000000 2017-12-31 03:37:35  sell  1.000000e-08    0.000148

[200 rows x 5 columns]

Mooncoin Volumes
----------------

**How far back in time does bleutrade order moonbtc_trades go?**

- 2017-12-31: for max moonbtc_trades of 200 orders, Timedelta('0 days 11:28:11'), which is less than a day

In [4]:
moonbtc_trades['datetime'].max() - moonbtc_trades['datetime'].min()

Timedelta('0 days 14:30:48')

**What is the average order size?**

- 2017-12-31: Average orders are BTC 0.006385 (buy at 2 satoshi) - 0.007357 (sell at 1 satoshi) which is about $85-100 USD

In [5]:
moonbtc_trades.groupby('side').mean()

             amount         price  btc_amount
side                                         
buy   308608.173097  2.000000e-08    0.006172
sell  874138.773931  1.000000e-08    0.008741

**What is the relative volume on the buy and sell sides of the market**

- 2017-12-31: There is currently 2.9x BTC / 5.8x MOON volume on the 1 satoshi sell side of the market

In [6]:
moonbtc_trades_sum = moonbtc_trades[['amount', 'btc_amount','side']].groupby('side').sum()
moonbtc_trades_sum

            amount  btc_amount
side                          
buy   2.160257e+07    0.432051
sell  1.136380e+08    1.136380

In [7]:
moonbtc_trades_sum.loc['sell'] / moonbtc_trades_sum.loc['buy']

amount        5.260394
btc_amount    2.630197
dtype: float64

**Can we plot a chart for the buy/sell volume history**

- 2017-12-31 - Apart from a massive 0.9 BTC sell spike at 7am, volumes remain low but about even on both sides of the market

In [8]:
moonbtc_trades_agg = moonbtc_trades.groupby([moonbtc_trades['datetime'].dt.hour, 'side'])['btc_amount'].sum()
moonbtc_trades_agg.unstack().plot()

Price
-----

** What is the price of mooncoin **

In [9]:
tickers = {
    'bleutrade': bleutrade.fetch_tickers(),
    'bittrex':   bittrex.fetch_tickers()
}
tickers['bleutrade']['MOON/BTC']

{'ask': 2e-08,
 'average': None,
 'baseVolume': 178496018.99498266,
 'bid': 1e-08,
 'change': None,
 'close': None,
 'datetime': '2017-12-31T18:25:10.000Z',
 'first': None,
 'high': 2e-08,
 'info': {'Ask': '0.00000002',
  'Average': '0.00000001',
  'BaseCurrency': 'Bitcoin',
  'BaseVolume': '2.15027971',
  'Bid': '0.00000001',
  'High': '0.00000002',
  'IsActive': 'true',
  'Last': '0.00000002',
  'Low': '0.00000001',
  'MarketCurrency': 'Mooncoin',
  'MarketName': 'MOON_BTC',
  'PrevDay': '0.00000002',
  'TimeStamp': '2017-12-31 18:25:10',
  'Volume': '178496018.99498266'},
 'last': 2e-08,
 'low': 1e-08,
 'open': None,
 'percentage': None,
 'quoteVolume': 2.15027971,
 'symbol': 'MOON/BTC',
 'timestamp': 1514744710000,
 'vwap': None}

In [10]:
def bid(market):       return market['bid']
def mid(market):       return (market['ask'] + market['bid'])/2
def spread(market):    return (market['ask'] - market['bid'])
def spread_pc(market): return spread(market)/mid(market)

In [11]:

price = {'bittrex': {}, 'bleutrade': {}}
price['bittrex']['BTC/USDT']    = bid(tickers['bittrex']['BTC/USDT'])
price['bittrex']['DOGE/BTC']    = bid(tickers['bittrex']['DOGE/BTC'])
price['bittrex']['ETH/BTC']     = bid(tickers['bittrex']['ETH/BTC'])
price['bleutrade']['DOGE/BTC']  = bid(tickers['bleutrade']['DOGE/BTC'])
price['bleutrade']['ETH/BTC']   = bid(tickers['bleutrade']['ETH/BTC'])
price['bleutrade']['MOON/BTC']  = bid(tickers['bleutrade']['MOON/BTC'])
price['bleutrade']['MOON/DOGE'] = bid(tickers['bleutrade']['MOON/DOGE'])
price['bleutrade']['MOON/ETH']  = bid(tickers['bleutrade']['MOON/ETH']) 

price['bleutrade']['MOON/ETH/BTC']     = price['bleutrade']['MOON/ETH']  * price['bleutrade']['ETH/BTC']
price['bleutrade']['MOON/DOGE/BTC']    = price['bleutrade']['MOON/DOGE'] * price['bleutrade']['DOGE/BTC']
price['bittrex']['MOON/ETH/BTC']       = price['bleutrade']['MOON/ETH']  * price['bittrex']['ETH/BTC']
price['bittrex']['MOON/DOGE/BTC']      = price['bleutrade']['MOON/DOGE'] * price['bittrex']['DOGE/BTC']
price['bittrex']['MOON/ETH/BTC/USDT']  = price['bleutrade']['MOON/ETH/BTC']  * price['bittrex']['BTC/USDT']
price['bittrex']['MOON/DOGE/BTC/USDT'] = price['bleutrade']['MOON/DOGE/BTC'] * price['bittrex']['BTC/USDT']
price['bittrex']['MOON/BTC/USDT']      = price['bleutrade']['MOON/BTC']      * price['bittrex']['BTC/USDT']                                                                                         

price                                                                                              

{'bittrex': {'BTC/USDT': 13830.0,
  'DOGE/BTC': 6.4e-07,
  'ETH/BTC': 0.05300001,
  'MOON/BTC/USDT': 0.0001383,
  'MOON/DOGE/BTC': 1.21600192e-08,
  'MOON/DOGE/BTC/USDT': 0.000165545361387,
  'MOON/ETH/BTC': 1.0600002e-08,
  'MOON/ETH/BTC/USDT': 0.0001447183647},
 'bleutrade': {'DOGE/BTC': 6.3e-07,
  'ETH/BTC': 0.05232045,
  'MOON/BTC': 1e-08,
  'MOON/DOGE': 0.01900003,
  'MOON/DOGE/BTC': 1.19700189e-08,
  'MOON/ETH': 2e-07,
  'MOON/ETH/BTC': 1.0464089999999999e-08}}

In [46]:
bleutrade_markets = pd.DataFrame(bleutrade.fetch_tickers()).transpose()[['ask', 'bid', 'last']]
bleutrade_markets['mid']    = (bleutrade_markets['ask'] + bleutrade_markets['bid']) / 2
bleutrade_markets['spread'] = (bleutrade_markets['ask'] - bleutrade_markets['bid']) / bleutrade_markets['bid']
bleutrade_markets = bleutrade_markets.filter(regex="MOON/|ETH/|DOGE/", axis=0)
bleutrade_markets

                 ask        bid       last         mid     spread
DOGE/BTC     6.4e-07    6.3e-07    6.3e-07    6.35e-07   0.015873
DOGE/ETH   1.264e-05  1.115e-05  1.264e-05  1.1895e-05   0.133632
ETH/BTC    0.0534218  0.0523219  0.0530004   0.0528718  0.0210228
ETH/DOGE       90000      65002      65002       77501   0.384573
MOON/BTC       2e-08      1e-08      2e-08     1.5e-08          1
MOON/DOGE  0.0239999      0.019      0.019      0.0215   0.263152
MOON/ETH     2.3e-07      2e-07    2.3e-07    2.15e-07       0.15

In [52]:
for symbol in bleutrade_markets.transpose():
    top  = re.split('/', symbol)[0]
    base = re.split('/', symbol)[-1]
    if base == 'BTC': continue
    if base in ['ETH', 'DOGE']:
        bleutrade_markets.loc[symbol+'/BTC'] = bleutrade_markets.loc[symbol] * bleutrade_markets.loc[base+'/BTC'] 
        bleutrade_markets.loc[symbol+'/BTC']['spread'] = bleutrade_markets.loc[symbol]['spread'] + bleutrade_markets.loc[base+'/BTC']['spread']         

print_full(bleutrade_markets)

                       ask          bid         last          mid     spread
DOGE/BTC           6.4e-07      6.3e-07      6.3e-07     6.35e-07   0.015873
DOGE/ETH         1.264e-05    1.115e-05    1.264e-05   1.1895e-05   0.133632
ETH/BTC          0.0534218    0.0523219    0.0530004    0.0528718  0.0210228
ETH/DOGE             90000        65002        65002        77501   0.384573
MOON/BTC             2e-08        1e-08        2e-08      1.5e-08          1
MOON/DOGE        0.0239999        0.019        0.019       0.0215   0.263152
MOON/ETH           2.3e-07        2e-07      2.3e-07     2.15e-07       0.15
DOGE/ETH/BTC   6.75252e-07  5.83389e-07  6.69924e-07   6.2891e-07   0.154655
ETH/DOGE/BTC        0.0576    0.0409513    0.0409513    0.0492131   0.400446
MOON/DOGE/BTC  1.53599e-08    1.197e-08    1.197e-08  1.36525e-08   0.279025
MOON/ETH/BTC    1.2287e-08  1.04644e-08  1.21901e-08  1.13674e-08   0.171023
